# 1. Загрузка данных из дз 1

In [4]:
SEED = 42

In [1]:
import pandas as pd

X_train = pd.read_csv("../data/X_train.csv")
X_val = pd.read_csv("../data/X_val.csv")
X_test = pd.read_csv("../data/X_test.csv")

y_train = pd.read_csv("../data/y_train.csv")
y_val = pd.read_csv("../data/y_val.csv")
y_test = pd.read_csv("../data/y_test.csv")

In [2]:
X_train.head(2)

,title_lemma,text_lemma
0,муравей оказываться способный обучать собрат,"ученый университет бристоль сообщать тот, обна..."
1,эстонский премьер пообещать решать проблема не...,"решать 10 год проблема лицо, не иметь гражданс..."


In [3]:
y_train.head(2)

,topic_encoded
0,6
1,4


# 2. Обучение word2vec-эмбеддингов с помощью библиотеки gensim

In [18]:
from gensim.models import Word2Vec
import string


def combine_features(X):
    return X["title_lemma"] + " " + X["text_lemma"]


X_train_combined = combine_features(X_train)

import re

punct_pattern = re.compile(f"[{re.escape(string.punctuation)}]")

# Удаляем знаки пунктуации и разбиваем на слова
sentences = [punct_pattern.sub("", text).split() for text in X_train_combined]

w2v_model = Word2Vec(
    sentences=sentences,
    vector_size=300,
    window=7,
    workers=10,
    min_count=10,
    sg=1,
    negative=5,
    epochs=20,
    seed=SEED,
)

- vector_size=300 - стандартный размер для хорошего качества векторов

- window=7 - берем чуть больше значения по умолчанию, чтобы учитывать тематическое сходство

- min_count=10 - игнорируем редкие слова, встречающиеся менее 10 раз

- sg=1 - Skip-gram

- negative=5 - для ускорения обучения

In [19]:
# 1. Проверка похожих слов для разных тематик

print("Похожие слова к 'президент':")
print(w2v_model.wv.most_similar("президент", topn=5))

print("\nПохожие слова к 'правительство':")
print(w2v_model.wv.most_similar("правительство", topn=5))

print("\nПохожие слова к 'компьютер':")
print(w2v_model.wv.most_similar("компьютер", topn=5))

print("\nПохожие слова к 'интернет':")
print(w2v_model.wv.most_similar("интернет", topn=5))

# 2. Проверка doesn't match
print("\nТест на выявление лишнего слова:")

tests = [
    ["россия", "франция", "германия", "компьютер"],
    ["чай", "кофе", "молоко", "телефон"],
    ["машина", "автомобиль", "велосипед", "президент"],
    ["врач", "медицина", "больница", "космос"],
]

for test in tests:
    try:
        odd_word = w2v_model.wv.doesnt_match(test)
        print(f"Из слов {test} лишнее слово: {odd_word}")
    except KeyError as e:
        print(f"Ошибка: некоторые слова отсутствуют в словаре - {e}")

Похожие слова к 'президент':
[('президентский', 0.5867680907249451), ('путин', 0.5542789101600647), ('премьерминистр', 0.5025945901870728), ('экспрезидент', 0.4944521486759186), ('глава', 0.488680362701416)]

Похожие слова к 'правительство':
[('премьерминистр', 0.6112547516822815), ('власть', 0.5802761912345886), ('премьер', 0.5524988174438477), ('кабмин', 0.5117477178573608), ('парламент', 0.5070345997810364)]

Похожие слова к 'компьютер':
[('ноутбук', 0.6030617356300354), ('предустановить', 0.5567210912704468), ('macintosh', 0.5512275099754333), ('планшетный', 0.5393248200416565), ('компьютерный', 0.5378121733665466)]

Похожие слова к 'интернет':
[('сеть', 0.5310444831848145), ('видеофайл', 0.4863573908805847), ('пользователь', 0.48340561985969543), ('видеореклама', 0.47732746601104736), ('скачивание', 0.4699729382991791)]

Тест на выявление лишнего слова:
Из слов ['россия', 'франция', 'германия', 'компьютер'] лишнее слово: компьютер
Из слов ['чай', 'кофе', 'молоко', 'телефон'] лишне

# 3. Установка предобученных эмбеддингов

- RusVectores

In [20]:
import urllib.request

urllib.request.urlretrieve(
    "https://rusvectores.org/static/models/rusvectores4/ruwikiruscorpora/ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz",
    "ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz",
)

('ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz',
 <http.client.HTTPMessage at 0x401a05dc0>)

In [21]:
import gensim

model_path = "ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz"
model_ru = gensim.models.KeyedVectors.load_word2vec_format(model_path)

- Navec

In [24]:
urllib.request.urlretrieve(
    "https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar",
    "navec_news_v1_1B_250K_300d_100q.tar",
)

('navec_news_v1_1B_250K_300d_100q.tar',
 <http.client.HTTPMessage at 0x4827ffa40>)

In [25]:
from navec import Navec

path = "navec_news_v1_1B_250K_300d_100q.tar"
navec = Navec.load(path)

# 4. Обучение LogisticRegression

- w2v

In [41]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


def get_document_vector(text, model, vector_size=300):
    words = punct_pattern.sub("", text).split()
    vector = np.zeros(vector_size)
    count = 0

    for word in words:
        try:
            vector += model.wv[word]
            count += 1
        except KeyError:
            continue

    if count > 0:
        vector /= count

    return vector


X_train_vectors = np.array(
    [get_document_vector(text, w2v_model) for text in combine_features(X_train)]
)

X_val_vectors = np.array(
    [get_document_vector(text, w2v_model) for text in combine_features(X_val)]
)

logreg = LogisticRegression(
    max_iter=1000,
    random_state=SEED,
)

logreg.fit(X_train_vectors, y_train.values.ravel())

y_val_pred = logreg.predict(X_val_vectors)
print(classification_report(y_val.values, y_val_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.89      0.23      0.36        35
           2       0.00      0.00      0.00         2
           3       0.54      0.22      0.31       200
           4       0.80      0.77      0.78      1444
           5       0.82      0.74      0.78       588
           6       0.63      0.55      0.59       747
           7       0.76      0.70      0.73      1208
           8       1.00      0.06      0.11        18
           9       0.00      0.00      0.00         9
          10       0.86      0.88      0.87      1456
          11       0.00      0.00      0.00         3
          12       0.78      0.84      0.81      3698
          13       0.84      0.82      0.83      1437
          14       0.77      0.53      0.63       174
          15       0.75      0.83      0.79      4342
          16       0.64      0.31      0.42       530
          17       0.97    

/Users/kulyaskin_mikhail/Library/Caches/pypoetry/virtualenvs/dl-nlp-7PEEyLUf-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/kulyaskin_mikhail/Library/Caches/pypoetry/virtualenvs/dl-nlp-7PEEyLUf-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/kulyaskin_mikhail/Library/Caches/pypoetry/virtualenvs/dl-nlp-7PEEyLUf-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 

- rusvec
- navec

In [84]:
# для rusvec определяем часть речи с помощью pymorphy2 (работает быстрее аналогов)
import pymorphy2

morph = pymorphy2.MorphAnalyzer()


def get_pos_pymorphy(word):
    parsed = morph.parse(word)[0]
    return parsed.tag.POS

In [85]:
def get_document_vector_rusvec(text, model, vector_size=300):
    words = punct_pattern.sub("", text).split()
    vector = np.zeros(vector_size)
    count = 0

    for word in words:
        try:
            pos = get_pos_pymorphy(word)

            word_with_pos = f"{word}_{pos}" if pos else word

            vector += model[word_with_pos]
            count += 1
        except KeyError:
            try:
                vector += model[word]
                count += 1
            except KeyError:
                continue

    if count > 0:
        vector /= count

    return vector


def get_document_vector_navec(text, model, vector_size=300):
    words = punct_pattern.sub("", text).split()
    vector = np.zeros(vector_size)
    count = 0

    for word in words:
        if word in model:
            vector += model[word]
            count += 1

    if count > 0:
        vector /= count

    return vector


X_train_rusvec = np.array(
    [get_document_vector_rusvec(text, model_ru) for text in combine_features(X_train)]
)

X_val_rusvec = np.array(
    [get_document_vector_rusvec(text, model_ru) for text in combine_features(X_val)]
)

X_train_navec = np.array(
    [get_document_vector_navec(text, navec) for text in combine_features(X_train)]
)

X_val_navec = np.array(
    [get_document_vector_navec(text, navec) for text in combine_features(X_val)]
)

logreg_rusvec = LogisticRegression(
    max_iter=1000,
    random_state=SEED,
)

logreg_navec = LogisticRegression(
    max_iter=1000,
    random_state=SEED,
)

logreg_rusvec.fit(X_train_rusvec, y_train.values.ravel())
logreg_navec.fit(X_train_navec, y_train.values.ravel())

y_val_pred_rusvec = logreg_rusvec.predict(X_val_rusvec)
y_val_pred_navec = logreg_navec.predict(X_val_navec)

print("\nОтчет о классификации rusvec:")
print(classification_report(y_val.values, y_val_pred_rusvec))
print("\nОтчет о классификации navec:")
print(classification_report(y_val.values, y_val_pred_navec))


Отчет о классификации rusvec:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.00      0.00      0.00        35
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00       200
           4       0.76      0.55      0.64      1444
           5       0.79      0.59      0.68       588
           6       0.60      0.45      0.51       747
           7       0.73      0.59      0.66      1208
           8       0.00      0.00      0.00        18
           9       0.00      0.00      0.00         9
          10       0.82      0.84      0.83      1456
          11       0.00      0.00      0.00         3
          12       0.73      0.82      0.77      3698
          13       0.74      0.76      0.75      1437
          14       0.73      0.13      0.22       174
          15       0.65      0.82      0.73      4342
          16       0.38      0.04      0.08       

/Users/kulyaskin_mikhail/Library/Caches/pypoetry/virtualenvs/dl-nlp-7PEEyLUf-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/kulyaskin_mikhail/Library/Caches/pypoetry/virtualenvs/dl-nlp-7PEEyLUf-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/kulyaskin_mikhail/Library/Caches/pypoetry/virtualenvs/dl-nlp-7PEEyLUf-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 

Лучшее качество у модели: w2v из gensim

# 5. Взвешивание через tf-idf

In [120]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()

train_texts = [punct_pattern.sub("", text) for text in combine_features(X_train)]
tfidf_vectorizer.fit(train_texts)


def get_document_vector_tfidf(text, model, tfidf_vectorizer, vector_size=300):
    clean_text = punct_pattern.sub("", text)
    words = clean_text.split()

    tfidf_vector = tfidf_vectorizer.transform([clean_text])[0]
    tfidf_weights = {}

    for word in set(words):
        try:
            idx = tfidf_vectorizer.vocabulary_.get(word)
            if idx is not None:
                tfidf_weights[word] = tfidf_vector[0, idx]
        except:
            continue

    vector = np.zeros(vector_size)
    total_weight = 0

    for word in words:
        try:
            weight = tfidf_weights.get(word, 0)
            if weight > 0:
                vector += model.wv[word] * weight
                total_weight += weight
        except KeyError:
            continue

    if total_weight > 0:
        vector /= total_weight

    return vector


X_train_vectors_tfidf = np.array(
    [
        get_document_vector_tfidf(text, w2v_model, tfidf_vectorizer)
        for text in combine_features(X_train)
    ]
)

X_val_vectors_tfidf = np.array(
    [
        get_document_vector_tfidf(text, w2v_model, tfidf_vectorizer)
        for text in combine_features(X_val)
    ]
)

logreg_tfidf = LogisticRegression(
    max_iter=1000,
    random_state=SEED,
)

logreg_tfidf.fit(X_train_vectors_tfidf, y_train.values.ravel())

y_val_pred_tfidf = logreg_tfidf.predict(X_val_vectors_tfidf)
print("Отчет о классификации с взвешиванием TF-IDF:")
print(classification_report(y_val.values, y_val_pred_tfidf))

print("\nОтчет о классификации без взвешивания TF-IDF (простое усреднение):")
print(classification_report(y_val.values, y_val_pred))

Отчет о классификации с взвешиванием TF-IDF:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.54      0.20      0.29        35
           2       0.00      0.00      0.00         2
           3       0.40      0.13      0.20       200
           4       0.77      0.73      0.75      1444
           5       0.76      0.72      0.74       588
           6       0.57      0.52      0.55       747
           7       0.73      0.64      0.69      1208
           8       0.00      0.00      0.00        18
           9       1.00      0.11      0.20         9
          10       0.83      0.85      0.84      1456
          11       0.00      0.00      0.00         3
          12       0.75      0.81      0.78      3698
          13       0.81      0.78      0.80      1437
          14       0.66      0.52      0.58       174
          15       0.72      0.79      0.75      4342
          16       0.50      0.23   

/Users/kulyaskin_mikhail/Library/Caches/pypoetry/virtualenvs/dl-nlp-7PEEyLUf-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/kulyaskin_mikhail/Library/Caches/pypoetry/virtualenvs/dl-nlp-7PEEyLUf-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/kulyaskin_mikhail/Library/Caches/pypoetry/virtualenvs/dl-nlp-7PEEyLUf-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 

# 6. Сравнение качества на тестовой выборке

In [140]:
X_test = X_test.dropna()
y_test = y_test.loc[X_test.index]

In [142]:
X_test_vectors = np.array(
    [get_document_vector(text, w2v_model) for text in combine_features(X_test)]
)

X_test_vectors_tfidf = np.array(
    [
        get_document_vector_tfidf(text, w2v_model, tfidf_vectorizer)
        for text in combine_features(X_test)
    ]
)

X_test_rusvec = np.array(
    [get_document_vector_rusvec(text, model_ru) for text in combine_features(X_test)]
)

X_test_navec = np.array(
    [get_document_vector_navec(text, navec) for text in combine_features(X_test)]
)

y_test_pred = logreg.predict(X_test_vectors)
y_test_pred_tfidf = logreg_tfidf.predict(X_test_vectors_tfidf)
y_test_pred_rusvec = logreg_rusvec.predict(X_test_rusvec)
y_test_pred_navec = logreg_navec.predict(X_test_navec)

print("Отчет о классификации на тестовой выборке (простое усреднение):")
print(classification_report(y_test.values, y_test_pred))

print("\nОтчет о классификации на тестовой выборке с взвешиванием TF-IDF:")
print(classification_report(y_test.values, y_test_pred_tfidf))

print("\nОтчет о классификации на тестовой выборке (rusvec):")
print(classification_report(y_test.values, y_test_pred_rusvec))

print("\nОтчет о классификации на тестовой выборке (navec):")
print(classification_report(y_test.values, y_test_pred_navec))

Отчет о классификации на тестовой выборке (простое усреднение):
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.89      0.24      0.37        34
           2       0.00      0.00      0.00         2
           3       0.68      0.27      0.39       200
           4       0.80      0.79      0.80      1445
           5       0.83      0.74      0.78       588
           6       0.68      0.59      0.63       747
           7       0.75      0.70      0.72      1209
           8       0.00      0.00      0.00        18
           9       0.00      0.00      0.00         9
          10       0.86      0.87      0.86      1455
          11       0.00      0.00      0.00         3
          12       0.79      0.84      0.81      3696
          13       0.84      0.83      0.84      1438
          14       0.71      0.52      0.60       173
          15       0.75      0.82      0.79      4342
          16     

/Users/kulyaskin_mikhail/Library/Caches/pypoetry/virtualenvs/dl-nlp-7PEEyLUf-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/kulyaskin_mikhail/Library/Caches/pypoetry/virtualenvs/dl-nlp-7PEEyLUf-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/kulyaskin_mikhail/Library/Caches/pypoetry/virtualenvs/dl-nlp-7PEEyLUf-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 